In [23]:
import matplotlib.pyplot as plt 
from PIL import Image
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [24]:
!pwd

/root/code/cyp-chasseray/deepfake-hunters/notebooks


In [25]:
image = Image.open(f"../raw_data/easy_1_1110.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/easy_1_1110.jpg'

In [26]:
image.size

(600, 600)

In [27]:
numpydata = np.array(image)

In [28]:
numpydata.shape

(600, 600, 3)

In [31]:
def load_data():
    X, y = [], []
    path = '../raw_data/fake'
    images = [f for f in listdir(path) if isfile(join(path, f))]
    for image in images:
        c_path = os.path.join(path, image)
        X.append(imread(c_path)[:, :, :3])
        y.append(1)
    path = '../raw_data/real'
    images = [f for f in listdir(path) if isfile(join(path, f))]
    for image in images:
        c_path = os.path.join(path, image)
        X.append(imread(c_path)[:, :, :3])
        y.append(0)
    c = list(zip(X, y))
    np.random.shuffle(c)
    X, y = zip(*c)
    return np.array(X), np.array(y)

In [32]:
X, y = load_data()

array([0, 1, 1, ..., 0, 1, 1])

In [37]:
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def initialize_model():
    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(600,600,3)))
    ### First Convolution & MaxPooling
    model.add(layers.Conv2D(16, (4,4), input_shape=(76, 78, 1), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    ### Second Convolution & MaxPooling
    model.add(layers.Conv2D(32, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    ### Third Convolution & MaxPooling
    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    ### Fourth Convolution & MaxPooling
    model.add(layers.Conv2D(64, (2,2), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    ### Flattening
    model.add(layers.Flatten())
    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(10, activation='relu')) 
    ### Last layer - Classification Layer with 10 outputs corresponding to 10 digits
    model.add(layers.Dense(1, activation='sigmoid'))
    
    ### Model compilation
    model.compile(optimizer = "adam",
                  loss = "binary_crossentropy",
                  metrics="accuracy")
    return model

2022-11-29 10:09:50.616347: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 10:09:51.878278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 10:09:51.878434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 10:09:52.057583: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-29 10:09:56.501872: W tensorflow/stream_executor/platform/de

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [39]:
model = initialize_model()

2022-11-29 10:13:50.062517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-29 10:13:50.062620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 10:13:50.062674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-29 10:13:50.062720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-29 10:13:50.062763: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 600, 600, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 597, 597, 16)      784       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 298, 298, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 296, 296, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 148, 148, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 146, 146, 64)      1

In [40]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5)

history = model.fit(X_train, y_train,
          batch_size = 16,
          epochs = 500,
          validation_split=0.3,
          callbacks = [es],
          verbose = 1)

Epoch 1/500


2022-11-29 10:15:01.545341: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1156680000 exceeds 10% of free system memory.


39/67 [================>.............] - ETA: 1:28 - loss: 0.7034 - accuracy: 0.4904

KeyboardInterrupt: 